In [1]:
import warnings
import datetime as dt
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [2]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

,a,z,symbol,tf,datetime,time,open,high,low,close,tick_volume,spread,real_volume
0,1,-99998,WIN$,M5,2021-11-25 12:30:00,1637843400,153184.0,153235.0,153113.0,153199.0,30898,1,108074
1,2,-99997,WIN$,M5,2021-11-25 12:35:00,1637843700,153206.0,153465.0,153163.0,153163.0,50380,1,186621
2,3,-99996,WIN$,M5,2021-11-25 12:40:00,1637844000,153163.0,153307.0,153055.0,153235.0,40566,1,137131
3,4,-99995,WIN$,M5,2021-11-25 12:45:00,1637844300,153242.0,153580.0,153170.0,153522.0,56134,1,202212
4,5,-99994,WIN$,M5,2021-11-25 12:50:00,1637844600,153522.0,153551.0,153235.0,153242.0,37612,1,126769


In [3]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['M5','M15','H1'])
Query.head()

,a,z,symbol,tf,datetime,time,open,high,low,close,tick_volume,spread,real_volume
0,1,-99998,WIN$,M5,2021-11-25 12:30:00,1637843400,153184.0,153235.0,153113.0,153199.0,30898,1,108074
1,2,-99997,WIN$,M5,2021-11-25 12:35:00,1637843700,153206.0,153465.0,153163.0,153163.0,50380,1,186621
2,3,-99996,WIN$,M5,2021-11-25 12:40:00,1637844000,153163.0,153307.0,153055.0,153235.0,40566,1,137131
3,4,-99995,WIN$,M5,2021-11-25 12:45:00,1637844300,153242.0,153580.0,153170.0,153522.0,56134,1,202212
4,5,-99994,WIN$,M5,2021-11-25 12:50:00,1637844600,153522.0,153551.0,153235.0,153242.0,37612,1,126769


# Data

In [4]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time

    Src_by          = Src.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)
    Src['T']        = Src_by['Time'].rank().sub(1).astype(int)

    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

,A,Z,Symbol,TF,Datetime,Date,Time,T,Ticks,Volume,Price,Open,High,Low,Close,Change,HL
0,1,-99998,WIN$,M5,2021-11-25 12:30:00,2021-11-25,12:30:00,0,30898,108074,153199.0,153184.0,153235.0,153113.0,153199.0,15.0,122.0
1,2,-99997,WIN$,M5,2021-11-25 12:35:00,2021-11-25,12:35:00,1,50380,186621,153163.0,153206.0,153465.0,153163.0,153163.0,-43.0,302.0
2,3,-99996,WIN$,M5,2021-11-25 12:40:00,2021-11-25,12:40:00,2,40566,137131,153235.0,153163.0,153307.0,153055.0,153235.0,72.0,252.0
3,4,-99995,WIN$,M5,2021-11-25 12:45:00,2021-11-25,12:45:00,3,56134,202212,153522.0,153242.0,153580.0,153170.0,153522.0,280.0,410.0
4,5,-99994,WIN$,M5,2021-11-25 12:50:00,2021-11-25,12:50:00,4,37612,126769,153242.0,153522.0,153551.0,153235.0,153242.0,-280.0,316.0


In [5]:
def CALCULATIONS(Src, METHOD):
    Calc = pd.DataFrame(Src)
    Calc_by2 = Calc.groupby(['Symbol','TF'],        sort=0, group_keys=0)
    Calc_by3 = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    # ================ Full Day ================ #
    Calc['Day Open']            = Calc_by3['Open'].transform('first')           .reset_index(drop=1)
    Calc['Day High']            = Calc_by3['High'].transform('max')             .reset_index(drop=1)
    Calc['Day Low']             = Calc_by3['Low'].transform('min')              .reset_index(drop=1)
    Calc['Day Close']           = Calc_by3['Close'].transform('last')           .reset_index(drop=1)

    Calc['Day HL']              = Calc['Day High']  - Calc['Day Low']
    Calc['Day Chg']             = Calc['Day Close'] - Calc['Day Open']
    Calc['Day Chg Abs']         = Calc['Day Chg'].abs()

    if METHOD == 'ROLLING_DATA':
        Calc['Day HL Avg']      = Calc_by2['Day HL']      .expanding().mean()   .reset_index(drop=1)
        Calc['Day Chg Abs Avg'] = Calc_by2['Day Chg Abs'] .expanding().mean()   .reset_index(drop=1)

    if METHOD == 'TOTAL_DATA':
        Calc['Day HL Avg']      = Calc_by2['Day HL']      .transform('mean')    .reset_index(drop=1)
        Calc['Day Chg Abs Avg'] = Calc_by2['Day Chg Abs'] .transform('mean')    .reset_index(drop=1)

    Calc['Day HL > Avg']        = Calc['Day HL']      > Calc['Day HL Avg']
    Calc['Day Chg Abs > Avg']   = Calc['Day Chg Abs'] > Calc['Day Chg Abs Avg']


    # ================ Intra Day ================ #
    Calc['Chg Abs']             = Calc['Change'].abs()

    if METHOD == 'ROLLING_DATA':
        Calc['Chg Abs Avg']     = Calc_by2['Chg Abs'].expanding().mean()        .reset_index(drop=1)
        Calc['HL Avg']          = Calc_by2['HL']     .expanding().mean()        .reset_index(drop=1)

    if METHOD == 'TOTAL_DATA':
        Calc['Chg Abs Avg']     = Calc_by2['Chg Abs'].transform('mean')         .reset_index(drop=1)
        Calc['HL Avg']          = Calc_by2['HL']     .transform('mean')         .reset_index(drop=1)


    Calc['Chg Abs > Avg']       = Calc['Chg Abs'] > Calc['Chg Abs Avg']
    Calc['HL > Avg']            = Calc['HL']      > Calc['HL Avg']      
    return Calc

Calc1 = CALCULATIONS(Src, METHOD='ROLLING_DATA')
Calc2 = CALCULATIONS(Src, METHOD='TOTAL_DATA')

# Stats

In [6]:
def STATS(Calc, T, Col):
    pipe = []
    for TF in ['M5','M15','H1']:
        Df = Calc[(Calc['TF']==TF) & (Calc['T']==T) & (Calc[Col])]
        
        pipe.append({ 'TF':TF,
            'Day Chg Abs > Avg':  np.round(Df['Day Chg Abs > Avg'] .mean() *100, 0).astype(int), 
            'Day HL > Avg':       np.round(Df['Day HL > Avg']      .mean() *100, 0).astype(int), 
        })
    pass
    return pd.DataFrame(pipe)

In [7]:
Sts11 = STATS(Calc1, T=0, Col='Chg Abs > Avg')
Sts11

,TF,Day Chg Abs > Avg,Day HL > Avg
0,M5,35,29
1,M15,37,31
2,H1,40,35


In [8]:
Sts12 = STATS(Calc1, T=0, Col='HL > Avg')
Sts12

,TF,Day Chg Abs > Avg,Day HL > Avg
0,M5,35,28
1,M15,35,31
2,H1,40,41


In [9]:
Sts21 = STATS(Calc2, T=0, Col='Chg Abs > Avg')
Sts21

,TF,Day Chg Abs > Avg,Day HL > Avg
0,M5,41,42
1,M15,44,46
2,H1,46,49


In [10]:
Sts22 = STATS(Calc2, T=0, Col='HL > Avg')
Sts22

,TF,Day Chg Abs > Avg,Day HL > Avg
0,M5,40,41
1,M15,42,45
2,H1,47,55


# Snippets

In [11]:
def CALCULATIONS(Src, METHOD):
    Calc = pd.DataFrame(Src)
    Calc_by2 = Calc.groupby(['Symbol','TF'],        sort=0, group_keys=0)
    Calc_by3 = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    # ================ Full Day ================ #
    Calc['Day Open']            = Calc_by3['Open'].transform('first')           .reset_index(drop=1)
    Calc['Day High']            = Calc_by3['High'].transform('max')             .reset_index(drop=1)
    Calc['Day Low']             = Calc_by3['Low'].transform('min')              .reset_index(drop=1)
    Calc['Day Close']           = Calc_by3['Close'].transform('last')           .reset_index(drop=1)

    Calc['Day HL']              = Calc['Day High']  - Calc['Day Low']
    Calc['Day Chg']             = Calc['Day Close'] - Calc['Day Open']
    Calc['Day Chg Abs']         = Calc['Day Chg'].abs()

    if METHOD == 'ROLLING_DATA':
        Calc['Day HL Avg']      = Calc_by2['Day HL']      .expanding().mean()   .reset_index(drop=1)
        Calc['Day Chg Abs Avg'] = Calc_by2['Day Chg Abs'] .expanding().mean()   .reset_index(drop=1)

    if METHOD == 'TOTAL_DATA':
        Calc['Day HL Avg']      = Calc_by2['Day HL']      .transform('mean')    .reset_index(drop=1)
        Calc['Day Chg Abs Avg'] = Calc_by2['Day Chg Abs'] .transform('mean')    .reset_index(drop=1)

    Calc['Day HL > Avg']        = Calc['Day HL']      > Calc['Day HL Avg']
    Calc['Day Chg Abs > Avg']   = Calc['Day Chg Abs'] > Calc['Day Chg Abs Avg']


    # ================ Intra Day ================ #
    Calc['Chg Abs']             = Calc['Change'].abs()

    if METHOD == 'ROLLING_DATA':
        Calc['Chg Abs Avg']     = Calc_by2['Chg Abs'].expanding().mean()        .reset_index(drop=1)
        Calc['HL Avg']          = Calc_by2['HL']     .expanding().mean()        .reset_index(drop=1)

    if METHOD == 'TOTAL_DATA':
        Calc['Chg Abs Avg']     = Calc_by2['Chg Abs'].transform('mean')         .reset_index(drop=1)
        Calc['HL Avg']          = Calc_by2['HL']     .transform('mean')         .reset_index(drop=1)


    Calc['Chg Abs > Avg']       = Calc['Chg Abs'] > Calc['Chg Abs Avg']
    Calc['HL > Avg']            = Calc['HL']      > Calc['HL Avg']      
    return Calc


def STATS(Calc, T, Col):
    pipe = []
    for TF in ['M5','M15','H1']:
        Df = Calc[(Calc['TF']==TF) & (Calc['T']==T) & (Calc[Col])]
        
        pipe.append({ 'TF':TF,
            'Day Chg Abs > Avg':  np.round(Df['Day Chg Abs > Avg'] .mean() *100, 0).astype(int), 
            'Day HL > Avg':       np.round(Df['Day HL > Avg']      .mean() *100, 0).astype(int), 
        })
    pass
    return pd.DataFrame(pipe)